# Transform data with topic probabilities

Once you have selected a model that suits your data, you will then need to transform that data back on to your original data table. Then, you can pull posts by their probability in a certain topic. You will want to use Excel or Google Sheets for that, unless you learn the Pandas df.query() syntax, which is <i>relatively</i> easy. Excel filtering is a lot easier because you don't need to write any code, and it works with relatively large datasets. But working with Pandas below will certainly be much faster to get a good overview of the dataset. ChatGPT can give you a good run down on how to query a dataset with Pandas :). But make sure to indicate the DataFrame.query() method when you prompt it!

A topic model doesn't simply assign a topic label to each document/post. Instead, it gives you a probability distribution of topics for that document. That means each document will have a certain probability of belonging to each topic. 0.12, or about 12%, is not very much, but usually values are lower than this. But 0.35 - 0.45 can start to be rather indicative of a topic. Usually, I start with 0.6, or about 60% likelihood of beloning to that topic to start feeling out a topic. This is just a guideline, every dataset is different and you need to experiment with these thresholds. 

This notebook starts by selecting a given topic model, and loading it. It only works with one k at a time! So you should first evaluate the different models before running this notebook so you know on which topic model you would like to run this computation.

It is generally good practice to take note of a qualitative name for each topic, which you should create for yourself in a text or spreadsheet file. You should create a key that links the NUMBER of the topic to the NAME you created. You can use the visualisation to determine what to name each topic.

So topic 4 from the visualization is actually topic 3 in the output dataset. I know this is annoying, but you need to keep it in mind when interpreting your results.

First we will load the topic model and then for each of our documents, we will calculate the probability of its belonging to the k topics that the model has. 

Then, we will load our <b>original</b> dataset from which we created the docbins in the Corpus Linguistics notebook. Then we will run all the same filters (the Corpus Linguistics notebook dumps all blank posts because they cause an error).

Then, we will join the probabilitiees we calculated in earlier to our original dataset, and export that to Excel.

## Step 0: Import packages

In [44]:
import pandas as pd
import os

### Step 1: Load the topic model

You need to enter a few variables in; this should be familiar from the previous notebooks.

In [22]:
from drs_topic_model import *

# original data table path -- the one you used to make the docbins. Make sure a copy of it is in the same folder as this notebook.
original_data_path = "../../input_data/anxiety.xlsx"

# the column of the original data table where the text for the docbins is stored
text_col = "body"

# project save name -- make sure this is the same that you have been using; this is important for the naming pattern to find your topic model!
project_save_name = "anxiety"

#create a folder for the probabilities
if os.path.exists(f"../../output_data/{project_save_name}_doc_topic_probabilities"):
    pass
else:
    os.mkdir(f"../../output_data/{project_save_name}_doc_topic_probabilities")

# select the k corresponding to the topic model you feel best suits your data
model_k = 10

### Step 2: Compute document-wise topic probability distribution

Errors in the code below are likely due to issues in the above cell.

In [23]:
%%time

topic_model_path = "../../output_data/" + project_save_name + "_topic_models/topic_model_k_" + str(model_k) + "/topic_model_k_" + str(model_k) + ".lda_mdl"

lda = gensim.models.ldamulticore.LdaMulticore
mdl = lda.load(topic_model_path)

corpus_path = f"../../intermediate_data/{project_save_name}_topic_models_params/{project_save_name}.mm"
corpus = MmCorpus(corpus_path)

rows = dict()
for i, doc in enumerate(corpus):
    row_data = {k:v for k, v in zip(range(model_k), [0 for x in range(model_k)])}
    for k, p in mdl[doc]:
        k_lab = "topic_" + str(k + 1)
        row_data[k_lab] = p
        rows[i] = row_data
        
tX = pd.DataFrame.from_dict(rows, orient="index").fillna(0)

df = None

original_data_type = original_data_path.split(".")[-1]

print(original_data_type)
if original_data_type == "xlsx":
    df = pd.read_excel(original_data_path)
elif original_data_type == "csv":
    df = pd.read_csv(original_data_path)
elif original_data_type == "pqt":
    df = pd.read_parquet(original_data_path)

df = df[df[text_col].isna() == False]
if len(corpus) == df.shape[0]:
    df = df.reset_index()
    df = df.join(tX)
    outfile = f"../../output_data/{project_save_name}_doc_topic_probabilities/{project_save_name}_doc_topic_probability_{model_k}.xlsx"
    
    df.to_excel(outfile, index=False, engine="xlsxwriter")
else:
    print("Length of corpus and length of original data table are not equal. Please consult how you created the docbins, particularly whether any filters to the dataframe were applied before your created the docbins.")
    print("Aborting computation.")

xlsx
CPU times: user 37.5 s, sys: 601 ms, total: 38.1 s
Wall time: 38.8 s


### Step 3: Query Data

Using Pandas (our data table manipulation library), you can easily query the dataset we just created. You can use this to show the documents associated with a topic by using the correct syntax. You are using the ">" and "<" operators (greater than/less then), or the ">=" or "<=" operators (greater than or equal to/less than or equal to). 

Here is a sample query for all the rows with a topic 2 probability greater than or equal to 0.6 (60% belonging to topic 2):

    df.query("topic_5 > 0.6")
    
    # we can chain these functions following the syntax
    df.query("topic_5 > 0.6").sort_values("topic_5", ascending = False) # this sorts our data largest to smallest (flip with ascending = True)
    
    df.query("topic_5 > 0.6").sort_values("topic_5", ascending = False).head(20) # give us the top 20 posts with highest probability for topic_5
    
    # and, if we save the query to the variable "q", then we can also print out the posts with the code:
    list(q["caption"]) (or whatever column you want)

In [ ]:
q = df.query("topic_4 > 0.6").sort_values("topic_5", ascending = False).head(5)
list(q["caption"])